# VacationPy
---

## Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint 
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sundumbili,-29.1337,31.3975,72.43,90,90,7.56,ZA,1706484011
1,1,port-aux-francais,-49.3500,70.2167,40.87,66,92,32.35,TF,1706484011
2,2,adamstown,-25.0660,-130.1015,76.14,77,100,10.74,PN,1706484012
3,3,blackmans bay,-43.0167,147.3167,71.73,57,5,5.88,AU,1706484012
4,4,ushuaia,-54.8000,-68.3000,60.46,44,40,11.50,AR,1706484013


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
import warnings
warnings.filterwarnings("ignore")

city_column = 'City'
humidity_column = 'Humidity'
# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    hover_cols=["City", "Humidty"],
    color=city_column,
    size=humidity_column,
)


# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
ideal_temp = (city_data_df['Max Temp'] >= 60) & (city_data_df['Max Temp'] <= 80)
ideal_humidity = city_data_df['Humidity'] < 60
ideal_cloudiness = city_data_df['Cloudiness'] < 30

# Combine the criteria using the logical AND (&) operator
ideal_conditions = ideal_temp & ideal_humidity & ideal_cloudiness

# Narrow down the DataFrame based on the criteria
ideal_city_data_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display the resulting DataFrame
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,blackmans bay,-43.0167,147.3167,71.73,57,5,5.88,AU,1706484012
17,17,toumodi,6.5613,-5.0158,73.71,44,0,3.56,CI,1706484018
33,33,el bolson,-41.9667,-71.5167,73.36,26,4,7.67,AR,1706484032
54,54,brady,31.1352,-99.3351,65.25,25,0,6.91,US,1706484036
71,71,baiti,13.7450,-14.7482,79.45,11,28,12.08,SN,1706484038
96,96,valdivia,-39.8142,-73.2459,73.49,44,0,2.55,CL,1706484041
98,98,chonchi,-42.6128,-73.8092,63.12,53,1,3.29,CL,1706484041
164,164,diapaga,12.0708,1.7889,73.63,14,0,8.57,BF,1706484048
190,190,saint-louis,16.3333,-15.0000,72.43,12,8,10.22,SN,1706484051
214,214,san patricio,28.0170,-97.5169,69.75,40,0,7.11,US,1706484053


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(len(hotel_df))


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,blackmans bay,-43.0167,147.3167,71.73,57,5,5.88,AU,1706484012,
17,17,toumodi,6.5613,-5.0158,73.71,44,0,3.56,CI,1706484018,
33,33,el bolson,-41.9667,-71.5167,73.36,26,4,7.67,AR,1706484032,
54,54,brady,31.1352,-99.3351,65.25,25,0,6.91,US,1706484036,
71,71,baiti,13.7450,-14.7482,79.45,11,28,12.08,SN,1706484038,
96,96,valdivia,-39.8142,-73.2459,73.49,44,0,2.55,CL,1706484041,
98,98,chonchi,-42.6128,-73.8092,63.12,53,1,3.29,CL,1706484041,
164,164,diapaga,12.0708,1.7889,73.63,14,0,8.57,BF,1706484048,
190,190,saint-louis,16.3333,-15.0000,72.43,12,8,10.22,SN,1706484051,
214,214,san patricio,28.0170,-97.5169,69.75,40,0,7.11,US,1706484053,


In [6]:
len(hotel_df)

28

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

https://api.geoapify.com/v2/places?categories=public_transport.subway&bias=proximity:-0.07071648508463113,51.50848194136378&limit=20&apiKey=YOUR_API_KEY


In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key, 
    'categories': 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search


blackmans bay - nearest hotel: Villa Howden
toumodi - nearest hotel: Allou Hôtel
el bolson - nearest hotel: Hotel Amancay
brady - nearest hotel: Holiday Inn Express and Suites
baiti - nearest hotel: No hotel found
valdivia - nearest hotel: Esmeralda
chonchi - nearest hotel: Hotel Huildín
diapaga - nearest hotel: Le Gacilien
saint-louis - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
greymouth - nearest hotel: Recreation Hotel
maturin - nearest hotel: Hotel Rossi
calama - nearest hotel: Apart Hotel Geotel Calama
nouadhibou - nearest hotel: Hotel valencia
atemajac de brizuela - nearest hotel: Koanze Luxury Hotel & Spa
laredo - nearest hotel: Rialto Hotel
taoudenni - nearest hotel: No hotel found
odienne - nearest hotel: Hotel Frontière
codrington - nearest hotel: No hotel found
nguigmi - nearest hotel: Guest PAM
ndikinimeki - nearest hotel: No hotel found
andalgala - nearest h

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,blackmans bay,-43.0167,147.3167,71.73,57,5,5.88,AU,1706484012,Villa Howden
17,17,toumodi,6.5613,-5.0158,73.71,44,0,3.56,CI,1706484018,Allou Hôtel
33,33,el bolson,-41.9667,-71.5167,73.36,26,4,7.67,AR,1706484032,Hotel Amancay
54,54,brady,31.1352,-99.3351,65.25,25,0,6.91,US,1706484036,Holiday Inn Express and Suites
71,71,baiti,13.7450,-14.7482,79.45,11,28,12.08,SN,1706484038,No hotel found
96,96,valdivia,-39.8142,-73.2459,73.49,44,0,2.55,CL,1706484041,Esmeralda
98,98,chonchi,-42.6128,-73.8092,63.12,53,1,3.29,CL,1706484041,Hotel Huildín
164,164,diapaga,12.0708,1.7889,73.63,14,0,8.57,BF,1706484048,Le Gacilien
190,190,saint-louis,16.3333,-15.0000,72.43,12,8,10.22,SN,1706484051,No hotel found
214,214,san patricio,28.0170,-97.5169,69.75,40,0,7.11,US,1706484053,No hotel found


In [8]:
name_address

{'type': 'FeatureCollection', 'features': []}

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    hover_cols=["City", "Hotel Name"],
    color='Hotel Name',
    size='Humidity',
)

# Display the map
hotel_map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City)